# Popular Data Science Questions

In this project, we're going to work for a company that creates data science content like books, online articles, videos, and interactive text-based learning platforms. It will be our task to determine the best content that the company should create based on data from [Data Science Stack Exchange](https://datascience.stackexchange.com/). Here we can find what kind of data science content is more popular, and see what people who are new to programming want to know.

Instead of having to scrape the website for data, Stack Exchange provides a public database that we can use [here](https://data.stackexchange.com/datascience/query/new) called the Stack Exchange Data Explorer–SEDE. We'll be using the Data Science Stack Exchange site; it is an ideal candidate for data since it is focussed on our topic and has a lot of unanswered questions.

Spend some time exploring Stack Exchange if you are not already familiar with it. After spending some time investigating DSSE, it looks like tags will be useful in categorizing our content.


